# Herramienta de Clasificación de Movimientos para STC

**Pipeline optimizado:**
1. Carga señales completas
2. Filtrado Butterworth + Notch
3. Separación por repeticiones (Train/ Val rep[2] / Test rep[5])
4. Normalización Z-score
5. Ventanas (500ms, 25% overlap)
6. Extracción features (ML) / Secuencias (DL)
7. Diferentes balances de clases en train
8. Entrenamiento y evaluación

In [ ]:
# ============================================================================
# 📦 SECCIÓN 1: INSTALACIÓN Y IMPORTS
# ============================================================================

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Instalar paquetes
!pip install imbalanced-learn pywavelets -q

# Imports
import os
import gc
import json
import pickle
import warnings
from pathlib import Path
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy import signal
from scipy.io import loadmat
import pywt

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from imblearn.over_sampling import ADASYN, SMOTE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow.keras.backend as K

# Definir función nombrada ANTES de construir modelos
def attention_sum(x):
    """Función nombrada para reemplazar lambda en atención"""
    return K.sum(x, axis=1)
# Registrar como custom object
keras.utils.get_custom_objects()['attention_sum'] = attention_sum

warnings.filterwarnings('ignore')
print("✅ Imports completados")

In [ ]:
# ============================================================================
# ⚙️ SECCIÓN 2: CONFIGURACIÓN
# ============================================================================

class Config:
    # Rutas
    BASE_DIR = Path('/content/drive/MyDrive')
    DATA_DIR = BASE_DIR / 'DB2_E1_only'
    TRAIN_DIR = DATA_DIR / 'train'
    SAVE_DIR = BASE_DIR / 'New_ML_DL_models_stc_optimized'

    # Parámetros de señal OPTIMIZADOS
    FS = 2000  # Hz
    WINDOW_SIZE_MS = 500  # 🔥 CAMBIADO: 300 → 500ms
    WINDOW_SIZE = int(FS * WINDOW_SIZE_MS / 1000)
    OVERLAP = 0.25  # 🔥 CAMBIADO: 0.5 → 0.25
    STEP_SIZE = int(WINDOW_SIZE * (1 - OVERLAP))
    N_CHANNELS = 12

    # Filtrado
    LOWCUT = 20
    HIGHCUT = 450
    NOTCH_FREQ = 50
    NOTCH_Q = 30

    # Clasificación binaria
    RISK_MOVEMENTS = [13, 14, 15, 16]
    SAFE_MOVEMENTS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17]

    # Repeticiones
    TRAIN_REPS = [1, 3, 4, 6]
    VAL_REPS = [2]
    TEST_REPS = [5]

    # DL params OPTIMIZADOS
    BATCH_SIZE = 64  # 🔥 CAMBIADO: más grande
    EPOCHS = 100
    LEARNING_RATE = 0.0005
    DROPOUT = 0.35  # 🔥 CAMBIADO: 0.4 → 0.22
    L2_REG = 0.001

    RANDOM_STATE = 42

cfg = Config()

In [ ]:
# ============================================================================
# 🎯 SECCIÓN 3: SELECCIÓN DE MODELOS Y SUJETOS
# ============================================================================

# 👤 CONFIGURACIÓN DE SUJETOS
USE_ALL_SUBJECTS = True  # Cambia a False para usar lista específica
SELECTED_SUBJECTS = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]  # Lista de sujetos a usar (ignorada si USE_ALL_SUBJECTS=True)

# 🤖 CONFIGURACIÓN DE MODELOS
SELECTED_MODELS = ['1', '2', '3', '4']  # Todos los modelos

AVAILABLE_MODELS = {
    '1': {'name': 'EnsembleSubspaceKNN', 'display': 'Ensemble KNN', 'type': 'ml'},
    '2': {'name': 'RandomForest', 'display': 'RF', 'type': 'ml'},
    '3': {'name': 'CNN_LSTM_Attention', 'display': 'CNN+LSTM+Attention', 'type': 'dl'},
    '4': {'name': 'BiLSTM_Attention', 'display': 'BiLSTM+Attention', 'type': 'dl'},
}

ML_MODELS = [m for m in SELECTED_MODELS if AVAILABLE_MODELS[m]['type'] == 'ml']
DL_MODELS = [m for m in SELECTED_MODELS if AVAILABLE_MODELS[m]['type'] == 'dl']
NEED_FEATURES = len(ML_MODELS) > 0

# ====== SELECCIÓN DE TÉCNICAS DE BALANCEO ======
# Para ML: 'none', 'adasyn', 'smote'
# Para DL: 'none', 'augment_only', 'focal_loss', 'focal_loss+augment'

ML_BALANCE_TECHNIQUE = 'adasyn'  # Cambiar aquí
DL_BALANCE_TECHNIQUE = 'augment_only'  # Cambiar aquí

# Validación
assert ML_BALANCE_TECHNIQUE in ['none', 'adasyn', 'smote'], "Técnica ML inválida"
assert DL_BALANCE_TECHNIQUE in ['none', 'augment_only', 'focal_loss', 'focal_loss+augment'], "Técnica DL inválida"

print(f"📊 Configuración:")
print(f"   Sujetos: {'TODOS' if USE_ALL_SUBJECTS else SELECTED_SUBJECTS}")
print(f"   Modelos ML: {ML_MODELS}")
print(f"   Modelos DL: {DL_MODELS}")
print(f"   Técnica balanceo ML: {ML_BALANCE_TECHNIQUE}")
print(f"   Técnica balanceo DL: {DL_BALANCE_TECHNIQUE}")

In [ ]:
# ============================================================================
# 📁 SECCIÓN 4: CREAR DIRECTORIOS
# ============================================================================

def create_run_dirs(cfg, tag=None):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_id = f"{ts}" + (f"_{tag}" if tag else "")
    cfg.RUN_ID = run_id
    cfg.RUN_DIR = cfg.SAVE_DIR / run_id
    cfg.PLOTS_DIR = cfg.RUN_DIR / "plots"
    cfg.ARTIFACTS_DIR = cfg.RUN_DIR / "artifacts"
    for d in (cfg.SAVE_DIR, cfg.RUN_DIR, cfg.PLOTS_DIR, cfg.ARTIFACTS_DIR):
        d.mkdir(parents=True, exist_ok=True)
    return cfg

tag = f"ML-{ML_BALANCE_TECHNIQUE}_DL-{DL_BALANCE_TECHNIQUE}"
create_run_dirs(cfg, tag=tag)
print(f"✅ Directorios creados: {cfg.RUN_DIR}")

In [ ]:
# ============================================================================
# 🔧 SECCIÓN 5: FUNCIONES DE FILTRADO
# ============================================================================

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data, axis=0)

def notch_filter(data, freq, fs, Q=30):
    b, a = signal.iirnotch(freq, Q, fs)
    return signal.filtfilt(b, a, data, axis=0)

def apply_filters(emg_data):
    filtered = butter_bandpass_filter(emg_data, cfg.LOWCUT, cfg.HIGHCUT, cfg.FS)
    filtered = notch_filter(filtered, cfg.NOTCH_FREQ, cfg.FS, cfg.NOTCH_Q)
    return filtered

print("✅ Funciones de filtrado definidas")

In [ ]:
# ============================================================================
# 📂 SECCIÓN 6: CARGA Y FILTRADO DE DATOS
# ============================================================================

def load_and_filter_data():
    """
    Carga y filtra señales EMG según configuración de sujetos
    """
    # Obtener todos los archivos .mat
    all_mat_files = sorted(cfg.TRAIN_DIR.glob('S*.mat'))

    # 🔥 FILTRAR SUJETOS SEGÚN CONFIGURACIÓN
    if USE_ALL_SUBJECTS:
        mat_files = all_mat_files
        print(f"\n📂 Cargando TODOS los sujetos disponibles...")
    else:
        mat_files = []
        for subj_id in SELECTED_SUBJECTS:
            file_path = cfg.TRAIN_DIR / f'S{subj_id}_E1_A1.mat'
            if file_path.exists():
                mat_files.append(file_path)
            else:
                print(f"⚠️  Advertencia: S{subj_id} no encontrado")

        mat_files = sorted(mat_files)
        print(f"\n📂 Cargando {len(mat_files)} sujetos seleccionados: {SELECTED_SUBJECTS}")

    if len(mat_files) == 0:
        raise ValueError("❌ No se encontraron archivos de sujetos")

    print(f"   Total de archivos a procesar: {len(mat_files)}")

    data_dict = {}

    for i, mat_file in enumerate(mat_files, 1):
        subj_id = mat_file.stem.split('_')[0]  # Extrae 'S1', 'S2', etc.

        # Cargar archivo .mat
        mat_data = loadmat(mat_file)
        emg_raw = mat_data['emg'].astype(np.float32)

        print(f"   [{i}/{len(mat_files)}] {subj_id}: {emg_raw.shape}")

        # Aplicar filtros
        emg_filtered = apply_filters(emg_raw)

        # Guardar en diccionario
        data_dict[subj_id] = {
            'emg': emg_filtered,
            'restimulus': mat_data['restimulus'].flatten(),
            'rerepetition': mat_data['rerepetition'].flatten()
        }

        # Liberar memoria
        del emg_raw, mat_data
        gc.collect()

    print(f"✅ {len(data_dict)} sujetos cargados y filtrados")
    return data_dict

data_dict = load_and_filter_data()

In [ ]:
# ============================================================================
# 🔀 SECCIÓN 7: DIVISIÓN POR REPETICIONES
# ============================================================================

def split_by_repetitions(data_dict):
    train_data, val_data, test_data = [], [], []

    for subj_id, data in data_dict.items():
        emg = data['emg']
        stimulus = data['restimulus']
        repetition = data['rerepetition']

        mask_train = np.isin(repetition, cfg.TRAIN_REPS)
        train_data.append({
            'emg': emg[mask_train],
            'stimulus': stimulus[mask_train],
            'repetition': repetition[mask_train],
            'subject': subj_id
        })

        mask_val = np.isin(repetition, cfg.VAL_REPS)
        val_data.append({
            'emg': emg[mask_val],
            'stimulus': stimulus[mask_val],
            'repetition': repetition[mask_val],
            'subject': subj_id
        })

        mask_test = np.isin(repetition, cfg.TEST_REPS)
        test_data.append({
            'emg': emg[mask_test],
            'stimulus': stimulus[mask_test],
            'repetition': repetition[mask_test],
            'subject': subj_id
        })

    def concatenate_data(data_list):
        return {
            'emg': np.vstack([d['emg'] for d in data_list]),
            'stimulus': np.hstack([d['stimulus'] for d in data_list]),
            'repetition': np.hstack([d['repetition'] for d in data_list])
        }

    return {
        'train': concatenate_data(train_data),
        'val': concatenate_data(val_data),
        'test': concatenate_data(test_data)
    }

split_data = split_by_repetitions(data_dict)
del data_dict
gc.collect()

print("\n📊 Datos divididos:")
for split in ['train', 'val', 'test']:
    print(f"   {split}: {split_data[split]['emg'].shape[0]:,} samples")

In [ ]:
# ============================================================================
# 🔧 SECCIÓN 8: NORMALIZACIÓN Z-SCORE
# ============================================================================

print("\n🔧 Normalizando señales...")

scaler = StandardScaler()
train_emg_scaled = scaler.fit_transform(split_data['train']['emg'])
val_emg_scaled = scaler.transform(split_data['val']['emg'])
test_emg_scaled = scaler.transform(split_data['test']['emg'])

with open(cfg.ARTIFACTS_DIR / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

split_data['train']['emg'] = train_emg_scaled.astype(np.float32)
split_data['val']['emg'] = val_emg_scaled.astype(np.float32)
split_data['test']['emg'] = test_emg_scaled.astype(np.float32)

del train_emg_scaled, val_emg_scaled, test_emg_scaled
gc.collect()
print("✅ Normalización completada")

In [ ]:
# ============================================================================
# 🔪 SECCIÓN 9: VENTANAS Y ETIQUETADO
# ============================================================================

def create_windows(emg, stimulus, window_size, step_size):
    n_samples = emg.shape[0]
    windows, labels = [], []

    for start in range(0, n_samples - window_size + 1, step_size):
        end = start + window_size
        window = emg[start:end]
        window_labels = stimulus[start:end]

        unique_labels, counts = np.unique(window_labels, return_counts=True)
        if 0 in unique_labels:
            continue

        majority_label = unique_labels[np.argmax(counts)]
        windows.append(window)
        labels.append(majority_label)

    return np.array(windows, dtype=np.float32), np.array(labels)

def binarize_labels(labels, risk_movements):
    return np.isin(labels, risk_movements).astype(np.int32)

print("\n🔪 Creando ventanas...")
windowed_data = {}

for split_name in ['train', 'val', 'test']:
    emg = split_data[split_name]['emg']
    stimulus = split_data[split_name]['stimulus']

    windows, labels = create_windows(emg, stimulus, cfg.WINDOW_SIZE, cfg.STEP_SIZE)
    binary_labels = binarize_labels(labels, cfg.RISK_MOVEMENTS)

    windowed_data[split_name] = {
        'windows': windows,
        'labels': binary_labels,
        'original_labels': labels
    }

    risk_count = (binary_labels == 1).sum()
    safe_count = (binary_labels == 0).sum()
    print(f"   {split_name.upper()}: {len(windows):,} ventanas | Risk: {risk_count:,} | Safe: {safe_count:,}")

del split_data
gc.collect()

In [ ]:
# ============================================================================
# ⚙️ SECCIÓN 10: EXTRACCIÓN DE FEATURES PARA ML
# ============================================================================

if NEED_FEATURES:
    def extract_time_domain_features(window):
        features = []
        for ch in range(window.shape[1]):
            sig = window[:, ch]
            mav = np.mean(np.abs(sig))
            wl = np.sum(np.abs(np.diff(sig)))
            zc = np.sum(np.diff(np.sign(sig)) != 0)
            diff_sig = np.diff(sig)
            ssc = np.sum(np.diff(np.sign(diff_sig)) != 0)
            rms = np.sqrt(np.mean(sig**2))
            features.extend([mav, wl, zc, ssc, rms])
        return np.array(features)

    def extract_frequency_features(window, fs=2000):
        features = []
        for ch in range(window.shape[1]):
            sig = window[:, ch]
            freqs = np.fft.rfftfreq(len(sig), 1/fs)
            psd = np.abs(np.fft.rfft(sig))**2
            total_power = np.sum(psd)
            if total_power > 0:
                mean_freq = np.sum(freqs * psd) / total_power
                cumsum = np.cumsum(psd)
                median_freq = freqs[np.searchsorted(cumsum, total_power/2)]
            else:
                mean_freq = median_freq = 0
            features.extend([mean_freq, median_freq])
        return np.array(features)

    def extract_wavelet_features(window, wavelet='db4', level=4):
        """Features wavelet (energía por nivel)"""
        features = []

        for ch in range(window.shape[1]):
            signal = window[:, ch]

            # DWT
            coeffs = pywt.wavedec(signal, wavelet, level=level)

            # Energía de cada nivel
            for coeff in coeffs:
                energy = np.sum(coeff**2)
                features.append(energy)

        return np.array(features)

    def extract_all_features(window):
        time_feats = extract_time_domain_features(window)
        freq_feats = extract_frequency_features(window)
        wave_feats = extract_wavelet_features(window)
        return np.concatenate([time_feats, freq_feats, wave_feats])

    print("\n⚙️ Extrayendo features para ML...")
    features_data = {}

    for split_name in ['train', 'val', 'test']:
        windows = windowed_data[split_name]['windows']
        labels = windowed_data[split_name]['labels']

        print(f"   {split_name.upper()}: {len(windows):,} ventanas...")
        features_list = []

        for i, window in enumerate(windows):
            features = extract_all_features(window)
            features_list.append(features)
            if (i+1) % 10000 == 0:
                print(f"      {i+1:,}/{len(windows):,}")

        features_array = np.array(features_list, dtype=np.float32)
        features_data[split_name] = {
            'features': features_array,
            'labels': labels
        }

        print(f"   ✅ Features shape: {features_array.shape}")

    del features_list
    gc.collect()

In [ ]:
# ============================================================================
# 🔄 SECCIÓN 11: BALANCEO DE CLASES PARA ML
# ============================================================================

if NEED_FEATURES:
    X_train = features_data['train']['features']
    y_train = features_data['train']['labels']

    original_safe = (y_train == 0).sum()
    original_risk = (y_train == 1).sum()

    if ML_BALANCE_TECHNIQUE == 'none':
        print("\n🔄 ML: Sin balanceo (none)")
        print(f"   Safe={original_safe:,} | Risk={original_risk:,}")
        # No hacemos nada, usamos los datos originales
        balanced_safe = original_safe
        balanced_risk = original_risk

    elif ML_BALANCE_TECHNIQUE == 'adasyn':
        print("\n🔄 ML: Aplicando ADASYN...")
        print(f"   Antes: Safe={original_safe:,} | Risk={original_risk:,}")
        adasyn = ADASYN(random_state=cfg.RANDOM_STATE, n_neighbors=5)
        X_train, y_train = adasyn.fit_resample(X_train, y_train)
        balanced_safe = (y_train == 0).sum()
        balanced_risk = (y_train == 1).sum()
        print(f"   Después: Safe={balanced_safe:,} | Risk={balanced_risk:,}")

    elif ML_BALANCE_TECHNIQUE == 'smote':
        print("\n🔄 ML: Aplicando SMOTE...")
        print(f"   Antes: Safe={original_safe:,} | Risk={original_risk:,}")
        smote = SMOTE(random_state=cfg.RANDOM_STATE, k_neighbors=5)
        X_train, y_train = smote.fit_resample(X_train, y_train)
        balanced_safe = (y_train == 0).sum()
        balanced_risk = (y_train == 1).sum()
        print(f"   Después: Safe={balanced_safe:,} | Risk={balanced_risk:,}")

    # Actualizar features_data con los datos balanceados (o sin balancear)
    features_data['train']['features'] = X_train.astype(np.float32)
    features_data['train']['labels'] = y_train

    del X_train, y_train
    gc.collect()

In [ ]:
# ============================================================================
# 📊 SECCIÓN 11.5: VISUALIZACIÓN DISTRIBUCIÓN BINARIA ANTES/DESPUÉS
# ============================================================================

if NEED_FEATURES and ML_BALANCE_TECHNIQUE != 'none':
    print("\n📊 Creando gráfica de distribución...")

    fig, ax = plt.subplots(figsize=(10, 6))

    categories = ['Antes del balanceo', 'Después del balanceo']
    safe_counts = [original_safe, balanced_safe]
    risk_counts = [original_risk, balanced_risk]

    x = np.arange(len(categories))
    width = 0.35

    bars1 = ax.bar(x - width/2, safe_counts, width, label='Safe', color='#2ecc71', alpha=0.8)
    bars2 = ax.bar(x + width/2, risk_counts, width, label='Risk', color='#e74c3c', alpha=0.8)

    ax.set_ylabel('Cantidad de muestras', fontsize=12, fontweight='bold')
    ax.set_title(f'Distribución de Clases - ML ({ML_BALANCE_TECHNIQUE.upper()})', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(categories)
    ax.legend(fontsize=10)
    ax.grid(axis='y', alpha=0.3)

    # Agregar valores en las barras
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height):,}',
                    ha='center', va='bottom', fontsize=10, fontweight='bold')

    plt.tight_layout()
    plt.savefig(cfg.PLOTS_DIR / 'class_distribution_balance.png', dpi=200, bbox_inches='tight')
    plt.show()

    print("✅ Gráfica guardada")

In [ ]:
# ============================================================================
# 🎯 SECCIÓN 12: FOCAL LOSS PARA DL
# ============================================================================

def focal_loss(gamma=2.0, alpha=0.75):
    """
    🔥 Focal Loss para manejar desbalance de clases
    gamma: factor de enfoque (mayor = más peso a ejemplos difíciles)
    alpha: peso para clase positiva
    """
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)

        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)

        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        focal_weight = K.pow(1.0 - pt, gamma)

        focal_loss_value = alpha * focal_weight * cross_entropy

        return K.mean(focal_loss_value)

    return loss

print("✅ Focal Loss definido")

In [ ]:
# ============================================================================
# 🧠 SECCIÓN 13: ARQUITECTURA CNN+LSTM CON ATTENTION
# ============================================================================

def build_cnn_lstm_attention(input_shape, use_focal_loss=True):
    """🔥 Versión REDUCIDA para evitar overfitting"""
    inputs = layers.Input(shape=input_shape)

    # Conv Block 1 - Reducido
    x = layers.Conv1D(64, 3, activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(cfg.L2_REG))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 3, activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Dropout(cfg.DROPOUT)(x)

    # Conv Block 2 - Reducido
    x = layers.Conv1D(256, 3, activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(256, 3, activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Dropout(cfg.DROPOUT)(x)

    # LSTM - REDUCIDAS
    x = layers.LSTM(64, return_sequences=True,
                    kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.Dropout(cfg.DROPOUT)(x)
    x = layers.LSTM(32, return_sequences=True,
                    kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)

    # Attention
    attention = layers.Dense(1, activation='tanh')(x)
    attention = layers.Flatten()(attention)
    attention = layers.Activation('softmax')(attention)
    attention = layers.RepeatVector(32)(attention)
    attention = layers.Permute([2, 1])(attention)

    x = layers.Multiply()([x, attention])
    x = layers.Lambda(attention_sum, name='attention_sum')(x)  # ✅ BUENO

    # Dense - Más regularización
    x = layers.Dense(32, activation='relu',
                     kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    # Seleccionar loss function según configuración
    loss_fn = focal_loss(gamma=2.5, alpha=0.75) if use_focal_loss else 'binary_crossentropy'

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=cfg.LEARNING_RATE),
        loss=loss_fn,
        metrics=['accuracy']
    )

    return model

# ============================================================================
# 🧠 SECCIÓN 14: BiLSTM CON ATTENTION (ANTI-OVERFITTING)
# ============================================================================

def build_bilstm_attention(input_shape, use_focal_loss=True):
    """🔥 Versión SIMPLIFICADA para evitar overfitting"""
    inputs = layers.Input(shape=input_shape)

    # BiLSTM - REDUCIDAS
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True,
                                         kernel_regularizer=regularizers.l2(cfg.L2_REG)))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(cfg.DROPOUT)(x)

    x = layers.Bidirectional(layers.LSTM(32, return_sequences=True,
                                         kernel_regularizer=regularizers.l2(cfg.L2_REG)))(x)
    x = layers.BatchNormalization()(x)

    # Attention
    attention = layers.Dense(1, activation='tanh')(x)
    attention = layers.Flatten()(attention)
    attention = layers.Activation('softmax')(attention)
    attention = layers.RepeatVector(64)(attention)
    attention = layers.Permute([2, 1])(attention)

    x = layers.Multiply()([x, attention])
    x = layers.Lambda(attention_sum, name='attention_sum')(x)  # ✅ BUENO

    # Dense con más regularización
    x = layers.Dense(32, activation='relu',
                     kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(16, activation='relu',
                     kernel_regularizer=regularizers.l2(cfg.L2_REG))(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    # Seleccionar loss function según configuración
    loss_fn = focal_loss(gamma=2.5, alpha=0.75) if use_focal_loss else 'binary_crossentropy'

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=cfg.LEARNING_RATE),
        loss=loss_fn,
        metrics=['accuracy']
    )

    return model

print("✅ BiLSTM con Attention definido")

In [ ]:
# ============================================================================
# 🔄 SECCIÓN 15: DATA AUGMENTATION PARA DL
# ============================================================================

def augment_signal(signal, augmentation_factor=0.7):
    """🔥 Data Augmentation MÁS AGRESIVO"""
    if np.random.rand() > augmentation_factor:
        return signal

    augmented = signal.copy()

    # Jittering más fuerte
    if np.random.rand() < 0.6:
        noise = np.random.normal(0, 0.025, augmented.shape)
        augmented = augmented + noise

    # Scaling más agresivo
    if np.random.rand() < 0.6:
        scale_factor = np.random.uniform(0.85, 1.15)
        augmented = augmented * scale_factor

    # Time warping más agresivo
    if np.random.rand() < 0.4:
        warp_factor = np.random.uniform(0.92, 1.08)
        new_length = int(len(augmented) * warp_factor)
        if new_length != len(augmented) and new_length > 0:
            indices = np.linspace(0, len(augmented)-1, new_length)
            augmented_warped = np.zeros((new_length, augmented.shape[1]))
            for ch in range(augmented.shape[1]):
                augmented_warped[:, ch] = np.interp(indices, np.arange(len(augmented)), augmented[:, ch])
            indices_back = np.linspace(0, new_length-1, len(augmented))
            for ch in range(augmented.shape[1]):
                augmented[:, ch] = np.interp(indices_back, np.arange(new_length), augmented_warped[:, ch])

    return augmented.astype(np.float32)

class DataGenerator(keras.utils.Sequence):
    """🔥 Generador con Data Augmentation"""
    def __init__(self, X, y, batch_size=64, augment=False, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.indices = np.arange(len(self.X))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, idx):
        indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch = self.X[indices]
        y_batch = self.y[indices]

        if self.augment:
            X_batch = np.array([augment_signal(x) for x in X_batch])

        return X_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

print("✅ Data Augmentation y Generador definidos")

In [ ]:
# ============================================================================
# 🔬 SECCIÓN 16: ENTRENAMIENTO MODELOS ML
# ============================================================================

ml_results = None

if len(ML_MODELS) > 0:
    print("\n" + "="*70)
    print("🔬 ENTRENAMIENTO MODELOS ML")
    print("="*70)

    X_train = features_data['train']['features']
    y_train = features_data['train']['labels']
    X_val = features_data['val']['features']
    y_val = features_data['val']['labels']
    X_test = features_data['test']['features']
    y_test = features_data['test']['labels']

    ml_results = {}

    # Modelo 1: Ensemble Subspace KNN
    if '1' in ML_MODELS:
        print("\n🔹 Entrenando Ensemble Subspace KNN...")
        base_knn = KNeighborsClassifier(n_neighbors=7, weights='distance', metric='euclidean')
        model = BaggingClassifier(
            estimator=base_knn,
            n_estimators=30,
            max_samples=0.7,
            max_features=0.8,
            random_state=cfg.RANDOM_STATE,
            n_jobs=-1
        )
        model.fit(X_train, y_train)

        y_pred_val = model.predict(X_val)
        y_pred_test = model.predict(X_test)

        ml_results['Ensemble_KNN'] = {
            'model': model,
            'val': {'y_true': y_val, 'y_pred': y_pred_val},
            'test': {'y_true': y_test, 'y_pred': y_pred_test}
        }
        # 💾 GUARDAR MODELO
        with open(cfg.ARTIFACTS_DIR / 'ensemble_knn.pkl', 'wb') as f:
            pickle.dump(model, f)
        print("   💾 Modelo guardado: ensemble_knn.pkl")
        print("   ✅ Completado")

    # Modelo 2: Random Forest
    if '2' in ML_MODELS:
        print("\n🔹 Entrenando Random Forest...")
        model = RandomForestClassifier(
            n_estimators=200,
            max_depth=30,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            random_state=cfg.RANDOM_STATE,
            n_jobs=-1
        )
        model.fit(X_train, y_train)

        y_pred_val = model.predict(X_val)
        y_pred_test = model.predict(X_test)

        ml_results['RandomForest'] = {
            'model': model,
            'val': {'y_true': y_val, 'y_pred': y_pred_val},
            'test': {'y_true': y_test, 'y_pred': y_pred_test}
        }
        # 💾 GUARDAR MODELO
        with open(cfg.ARTIFACTS_DIR / 'random_forest.pkl', 'wb') as f:
            pickle.dump(model, f)
        print("   💾 Modelo guardado: random_forest.pkl")
        print("   ✅ Completado")

# ============================================================================
# 🧠 SECCIÓN 17: ENTRENAMIENTO MODELOS DL
# ============================================================================

dl_results = None

if len(DL_MODELS) > 0:
    print("\n" + "="*70)
    print("🧠 ENTRENAMIENTO MODELOS DL")
    print("="*70)

    X_train = windowed_data['train']['windows']
    y_train = windowed_data['train']['labels'].reshape(-1, 1)
    X_val = windowed_data['val']['windows']
    y_val = windowed_data['val']['labels'].reshape(-1, 1)
    X_test = windowed_data['test']['windows']
    y_test = windowed_data['test']['labels'].reshape(-1, 1)

    print(f"\n📊 Shapes DL:")
    print(f"   X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"   X_val: {X_val.shape}, y_val: {y_val.shape}")
    print(f"   X_test: {X_test.shape}, y_test: {y_test.shape}")

    # Determinar configuración según técnica de balanceo DL
    use_focal_loss = DL_BALANCE_TECHNIQUE in ['focal_loss', 'focal_loss+augment']
    use_augmentation = DL_BALANCE_TECHNIQUE in ['augment_only', 'focal_loss+augment']

    print(f"\n🎯 Configuración DL:")
    print(f"   Loss: {'Focal Loss' if use_focal_loss else 'Binary Crossentropy'}")
    print(f"   Augmentation: {'Activado' if use_augmentation else 'Desactivado'}")

    # Generadores
    train_gen = DataGenerator(X_train, y_train, batch_size=cfg.BATCH_SIZE, augment=use_augmentation, shuffle=True)
    val_gen = DataGenerator(X_val, y_val, batch_size=cfg.BATCH_SIZE, augment=False, shuffle=False)

    # Callbacks
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=15,
        min_delta=0.001,
        restore_best_weights=True,
        mode='min',
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=8,
        min_lr=1e-7,
        verbose=1
    )

    dl_results = {}
    input_shape = (X_train.shape[1], X_train.shape[2])

    # Modelo 3: CNN+LSTM con Attention
    if '3' in DL_MODELS:
        print("\n🔹 Entrenando CNN+LSTM con Attention...")
        model = build_cnn_lstm_attention(input_shape, use_focal_loss=use_focal_loss)

        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=cfg.EPOCHS,
            callbacks=[early_stop, reduce_lr],
            verbose=1
        )

        # Predicciones
        y_pred_val = (model.predict(X_val, batch_size=cfg.BATCH_SIZE) > 0.5).astype(int)
        y_pred_test = (model.predict(X_test, batch_size=cfg.BATCH_SIZE) > 0.5).astype(int)

        dl_results['CNN_LSTM_Attention'] = {
            'model': model,
            'history': history.history,
            'val': {'y_true': y_val.flatten(), 'y_pred': y_pred_val.flatten()},
            'test': {'y_true': y_test.flatten(), 'y_pred': y_pred_test.flatten()}
        }

        # Guardar modelo
        #model.save(cfg.ARTIFACTS_DIR / 'cnn_lstm_attention.h5')
        model.save(cfg.ARTIFACTS_DIR / 'cnn_lstm_attention.keras')
        print("   ✅ Completado")

    # Modelo 4: BiLSTM con Attention
    if '4' in DL_MODELS:
        print("\n🔹 Entrenando BiLSTM con Attention...")
        model = build_bilstm_attention(input_shape, use_focal_loss=use_focal_loss)

        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=cfg.EPOCHS,
            callbacks=[early_stop, reduce_lr],
            verbose=1
        )

        # Predicciones
        y_pred_val = (model.predict(X_val, batch_size=cfg.BATCH_SIZE) > 0.5).astype(int)
        y_pred_test = (model.predict(X_test, batch_size=cfg.BATCH_SIZE) > 0.5).astype(int)

        dl_results['BiLSTM_Attention'] = {
            'model': model,
            'history': history.history,
            'val': {'y_true': y_val.flatten(), 'y_pred': y_pred_val.flatten()},
            'test': {'y_true': y_test.flatten(), 'y_pred': y_pred_test.flatten()}
        }

        # Guardar modelo
        #model.save(cfg.ARTIFACTS_DIR / 'bilstm_attention.h5')
        model.save(cfg.ARTIFACTS_DIR / 'bilstm_attention.keras')
        print("   ✅ Completado")

In [ ]:
# ============================================================================
# 🎯 SECCIÓN 18: THRESHOLD OPTIMIZATION
# ============================================================================

print("\n" + "="*70)
print("🎯 OPTIMIZACIÓN DE THRESHOLD")
print("="*70)

def optimize_threshold(y_true, y_proba):
    """🔥 Encuentra el threshold óptimo que maximiza F1"""
    best_threshold = 0.5
    best_f1 = 0

    for threshold in np.arange(0.3, 0.8, 0.05):
        y_pred = (y_proba >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred, zero_division=0)

        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold, best_f1

# Optimizar para cada modelo
optimized_thresholds = {}

if dl_results is not None:
    for model_name, results in dl_results.items():
        print(f"\n🔍 Optimizando threshold para {model_name}...")

        # Obtener probabilidades en validation set
        model = results['model']
        y_val_true = windowed_data['val']['labels']
        y_val_proba = model.predict(windowed_data['val']['windows'], batch_size=cfg.BATCH_SIZE).flatten()

        best_thresh, best_f1_val = optimize_threshold(y_val_true, y_val_proba)
        optimized_thresholds[model_name] = best_thresh

        print(f"   Threshold óptimo: {best_thresh:.2f}")
        print(f"   F1 en validation: {best_f1_val:.4f}")

        # Re-predecir en test con threshold óptimo
        y_test_proba = model.predict(windowed_data['test']['windows'], batch_size=cfg.BATCH_SIZE).flatten()
        y_test_pred_optimized = (y_test_proba >= best_thresh).astype(int)

        # Actualizar resultados
        dl_results[model_name]['test']['y_pred'] = y_test_pred_optimized
        dl_results[model_name]['optimized_threshold'] = best_thresh

print("\n✅ Threshold optimization completada")

In [ ]:
# ============================================================================
# 📊 SECCIÓN 19: EVALUACIÓN FINAL
# ============================================================================

def evaluate_model(y_true, y_pred, model_name, split='test'):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f"\n{'='*50}")
    print(f"{model_name} - {split.upper()}")
    print(f"{'='*50}")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1-Score:  {f1:.4f}")

    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1}

print("\n" + "="*70)
print("📊 EVALUACIÓN FINAL EN TEST SET")
print("="*70)

all_metrics = {}

if ml_results is not None:
    for model_name, results in ml_results.items():
        metrics = evaluate_model(
            results['test']['y_true'],
            results['test']['y_pred'],
            model_name,
            'test'
        )
        all_metrics[model_name] = metrics

if dl_results is not None:
    for model_name, results in dl_results.items():
        metrics = evaluate_model(
            results['test']['y_true'],
            results['test']['y_pred'],
            model_name,
            'test'
        )
        all_metrics[model_name] = metrics

In [ ]:
# ============================================================================
# 📈 SECCIÓN 20: VISUALIZACIÓN DE RESULTADOS
# ============================================================================

# Comparación de métricas
def plot_metrics_comparison(metrics_dict):
    df = pd.DataFrame(metrics_dict).T

    fig, ax = plt.subplots(figsize=(12, 6))
    df.plot(kind='bar', ax=ax, width=0.7, colormap='viridis')

    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_xlabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title('Comparación de Métricas en Test Set', fontsize=14, fontweight='bold')
    ax.set_ylim([0, 1])
    ax.legend(title='Métrica', fontsize=10)
    ax.grid(axis='y', alpha=0.3)

    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', fontsize=8)

    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(cfg.PLOTS_DIR / 'metrics_comparison.png', dpi=200, bbox_inches='tight')
    plt.show()

plot_metrics_comparison(all_metrics)

# Matrices de confusión
def plot_confusion_matrices(ml_results, dl_results):
    all_results = {}
    if ml_results:
        all_results.update(ml_results)
    if dl_results:
        all_results.update(dl_results)

    n_models = len(all_results)
    fig, axes = plt.subplots(1, n_models, figsize=(5*n_models, 4))

    if n_models == 1:
        axes = [axes]

    for i, (model_name, results) in enumerate(all_results.items()):
        y_true = results['test']['y_true']
        y_pred = results['test']['y_pred']

        cm = confusion_matrix(y_true, y_pred, normalize='true')

        sns.heatmap(cm, annot=True, fmt='.2%', cmap='Blues',
                    xticklabels=['Safe', 'Risk'],
                    yticklabels=['Safe', 'Risk'],
                    ax=axes[i], cbar_kws={'label': 'Proporción'})

        axes[i].set_title(f'{model_name}', fontweight='bold')
        axes[i].set_ylabel('True Label')
        axes[i].set_xlabel('Predicted Label')

    plt.tight_layout()
    plt.savefig(cfg.PLOTS_DIR / 'confusion_matrices.png', dpi=200, bbox_inches='tight')
    plt.show()

plot_confusion_matrices(ml_results, dl_results)

# Curvas ROC
def plot_roc_curves(ml_results, dl_results):
    plt.figure(figsize=(10, 8))

    all_results = {}
    if ml_results:
        all_results.update(ml_results)
    if dl_results:
        all_results.update(dl_results)

    for model_name, results in all_results.items():
        y_true = results['test']['y_true']
        y_pred = results['test']['y_pred']

        fpr, tpr, _ = roc_curve(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred)

        plt.plot(fpr, tpr, label=f'{model_name} (AUC={auc:.3f})', linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC=0.5)', linewidth=2)
    plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
    plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
    plt.title('Curvas ROC - Test Set', fontsize=14, fontweight='bold')
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(cfg.PLOTS_DIR / 'roc_curves.png', dpi=200, bbox_inches='tight')
    plt.show()

plot_roc_curves(ml_results, dl_results)

# Training history (DL)
if dl_results is not None:
    n_models = len(dl_results)
    fig, axes = plt.subplots(n_models, 2, figsize=(14, 4*n_models))

    if n_models == 1:
        axes = axes.reshape(1, -1)

    for i, (model_name, results) in enumerate(dl_results.items()):
        history = results['history']

        # Loss
        axes[i, 0].plot(history['loss'], label='Train Loss', linewidth=2)
        axes[i, 0].plot(history['val_loss'], label='Val Loss', linewidth=2)
        axes[i, 0].set_title(f'{model_name} - Loss', fontweight='bold')
        axes[i, 0].set_xlabel('Epoch')
        axes[i, 0].set_ylabel('Loss')
        axes[i, 0].legend()
        axes[i, 0].grid(alpha=0.3)

        # Accuracy
        axes[i, 1].plot(history['accuracy'], label='Train Acc', linewidth=2)
        axes[i, 1].plot(history['val_accuracy'], label='Val Acc', linewidth=2)
        axes[i, 1].set_title(f'{model_name} - Accuracy', fontweight='bold')
        axes[i, 1].set_xlabel('Epoch')
        axes[i, 1].set_ylabel('Accuracy')
        axes[i, 1].legend()
        axes[i, 1].grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(cfg.PLOTS_DIR / 'training_history.png', dpi=200, bbox_inches='tight')
    plt.show()

In [ ]:
# ============================================================================
# 💾 SECCIÓN 21: GUARDAR RESULTADOS
# ============================================================================

# Preparar descripción de técnicas para metadata
ml_technique_desc = {
    'none': 'Sin balanceo',
    'adasyn': 'ADASYN',
    'smote': 'SMOTE'
}[ML_BALANCE_TECHNIQUE]

dl_technique_desc = {
    'none': 'Binary Crossentropy',
    'augment_only': 'Binary Crossentropy + Data Augmentation',
    'focal_loss': 'Focal Loss (gamma=2.5, alpha=0.75)',
    'focal_loss+augment': 'Focal Loss (gamma=2.5, alpha=0.75) + Data Augmentation'
}[DL_BALANCE_TECHNIQUE]

metadata = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'config': {
        'window_size_ms': cfg.WINDOW_SIZE_MS,
        'overlap': cfg.OVERLAP,
        'batch_size': cfg.BATCH_SIZE,
        'dropout': cfg.DROPOUT,
        'learning_rate': cfg.LEARNING_RATE,
        'ml_balance_technique': ML_BALANCE_TECHNIQUE,
        'ml_balance_description': ml_technique_desc,
        'dl_balance_technique': DL_BALANCE_TECHNIQUE,
        'dl_balance_description': dl_technique_desc
    },
    'optimized_thresholds': optimized_thresholds,
    'results': all_metrics
}

with open(cfg.ARTIFACTS_DIR / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("\n✅ Metadata guardada")

# Tabla resumen
print("\n" + "="*70)
print("📋 RESUMEN FINAL")
print("="*70)
df_results = pd.DataFrame(all_metrics).T
df_results = df_results.round(4)
print(df_results)
df_results.to_csv(cfg.ARTIFACTS_DIR / 'results_summary.csv')

print(f"\n🎉 ENTRENAMIENTO COMPLETADO")
print(f"📁 Resultados guardados en: {cfg.RUN_DIR}")
print(f"📊 Gráficas en: {cfg.PLOTS_DIR}")
print(f"💾 Artefactos en: {cfg.ARTIFACTS_DIR}")
print(f"\n⚙️ Técnicas utilizadas:")
print(f"   ML: {ml_technique_desc}")
print(f"   DL: {dl_technique_desc}")